In [ ]:
import mysql.connector as sql
import easyocr
import re
import os
import pandas as pd

In [ ]:
mydb = sql.connect(host="localhost",
                   user="root",
                   password="#Saravanan27",
                   database= "bizcard_db" #If not create one 
                  )
mycursor = mydb.cursor(buffered=True)

In [ ]:
mycursor.execute("CREATE DATABASE bizcard_db")

In [ ]:
# TABLE CREATION
mycursor.execute('''CREATE TABLE IF NOT EXISTS business_card_data
                   (id INTEGER PRIMARY KEY AUTO_INCREMENT,
                    card_holder VARCHAR(255),
                    designation VARCHAR(255),
                    company_name VARCHAR(255),
                    email TEXT,
                    website TEXT,
                    mobile_number VARCHAR(50),
                    area TEXT,
                    city TEXT,
                    state TEXT,
                    pin_code VARCHAR(10)
                    )''')

mydb.commit()

In [ ]:
mycursor.execute("show tables")
mycursor.fetchall()

In [ ]:
mycursor.execute("select * from business_card_data")
mycursor.fetchall()

In [ ]:
reader = easyocr.Reader(["en"])

results = reader.readtext('Cards/Card.png')
#for (bbox,text,prob) in result: # To check what's inside 
    #print(text)   

In [ ]:
text_contents = [result[1] for result in results]
text_contents

In [ ]:
saved_img = os.getcwd()+ "\\" + "Cards"+ "\\"+ "Card1.png"
result = reader.readtext(saved_img,detail = 0,paragraph=False)

In [ ]:
result

In [ ]:
data = {"card_holder" : [],
        "designation" : [],
        "company_name" : [],
        "email" : [],
        "mobile_number" :[], 
        "website" : [],
        "area" : [],
        "city" : [],
        "state" : [],
        "pin_code" : [],
        }

def get_data(result):
    for ind,i in enumerate(result):
        
        # To get WEBSITE_URL
        if "www " in i.lower() or "www." in i.lower():
                data["website"].append(i)
        elif "WWW" in i:
                data["website"] = result[4] +"." + result[5]

        # To get EMAIL ID
        elif "@" in i:
                data["email"].append(i)

        # To get MOBILE NUMBER
        elif "-" in i:
                data["mobile_number"].append(i)
                if len(data["mobile_number"]) ==2:
                        data["mobile_number"] = " & ".join(data["mobile_number"])
                      
        elif ind == len(result) -1:
                data["company_name"].append(i)       

        # To get CARD HOLDER NAME
        elif ind == 0:
            data["card_holder"].append(i)

        # To get DESIGNATION
        elif ind == 1:
                data["designation"].append(i)

        # To get AREA
        if re.findall('^[0-9].+, [a-zA-Z]+',i):
                data["area"].append(i.split(',')[0])
        elif re.findall('[0-9] [a-zA-Z]+',i):
                data["area"].append(i)

        # To get CITY NAME
        match1 = re.findall('.+St , ([a-zA-Z]+).+', i)
        match2 = re.findall('.+St,, ([a-zA-Z]+).+', i)
        match3 = re.findall('^[E].*',i)
        if match1:
                data["city"].append(match1[0])
        elif match2:
                data["city"].append(match2[0])
        elif match3:
                data["city"].append(match3[0])

        # To get STATE
        state_match = re.findall('[a-zA-Z]{9} +[0-9]',i)
        if state_match:
                data["state"].append(i[:9])
        elif re.findall('^[0-9].+, ([a-zA-Z]+);',i):
                data["state"].append(i.split()[-1])
        if len(data["state"])== 2:
                data["state"].pop(0)        

        # To get PINCODE        
        if len(i)>=6 and i.isdigit():
                data["pin_code"].append(i)
        elif re.findall('[a-zA-Z]{9} +[0-9]',i):
                data["pin_code"].append(i[10:])

#Extract Company
#company = result[-1:]
#companies = '\n'.join(company)
#company_name= companies.replace('\n', ' ')
#company_name = data['company_name'].append(company_name)          
#get_data(result)

get_data(result)

def create_df(data):
            df = pd.DataFrame(data)
            return df
df = create_df(data)

#st.success("### Data Extracted!")
#st.write(df)

for i,row in df.iterrows():
#here %S means string values 
        sql = """INSERT INTO business_card_data(card_holder,designation,company_name,email,mobile_number,website,area,city,state,pin_code)
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        mycursor.execute(sql, tuple(row))
        # the connection is not auto committed by default, so we must commit to save our changes
        mydb.commit()
        #st.success("#### Uploaded to database successfully!")

In [ ]:
df